In [1]:
!pip install kaggle


In [0]:
!mkdir .kaggle

In [0]:
import json
token = {"username":"sushmita1794","key":"863998e4922d86a43de1845b384363d1"}

with open('/content/.kaggle/kaggle.json', 'w') as file: json.dump(token, file)


In [0]:
!chmod 600 /content/.kaggle/kaggle.json

In [0]:
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json

In [10]:
!kaggle config set -n path -v{/content}

- path is now set to: {/content}


In [11]:
!kaggle datasets download -d usdot/flight-delays

 97% 185M/191M [00:02<00:00, 75.3MB/s]
100% 191M/191M [00:02<00:00, 83.3MB/s]


In [12]:
!unzip /content/sample_data/flight-delays.zip

Archive:  /content/sample_data/flight-delays.zip
  inflating: airlines.csv            
  inflating: airports.csv            
  inflating: flights.csv             


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time

In [14]:
#load the csv file and use the first 100000 rows of data
df_use = pd.read_csv('flights.csv')
df = df_use.sample(n=100000).copy()
del df_use

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [15]:
df.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
1120964,2015,3,14,6,EV,3948,N15555,ORD,LAN,1642,1633.0,-9.0,14.0,1647.0,58.0,53.0,35.0,179,1822.0,4.0,1840,1826.0,-14.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2536963,2015,6,10,3,MQ,3280,N510MQ,ELP,ORD,610,608.0,-2.0,11.0,619.0,185.0,186.0,164.0,1236,1003.0,11.0,1015,1014.0,-1.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4034065,2015,9,8,2,WN,3076,N418WN,LIT,MDW,1155,1152.0,-3.0,7.0,1159.0,100.0,137.0,125.0,544,1404.0,5.0,1335,1409.0,34.0,0,0,NaN,34.0,0.0,0.0,0.0,0.0
4632793,2015,10,16,5,AA,1740,N559UW,13487,11057,1345,1350.0,5.0,12.0,1402.0,155.0,139.0,113.0,930,1655.0,14.0,1720,1709.0,-11.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4857038,2015,10,30,5,NK,425,N620NK,13930,11697,1819,2013.0,114.0,15.0,2028.0,179.0,170.0,148.0,1182,2356.0,7.0,2218,3.0,105.0,0,0,NaN,105.0,0.0,0.0,0.0,0.0


In [16]:
df.groupby('DEPARTURE_DELAY').size()

DEPARTURE_DELAY
-38.0      1
-36.0      1
-31.0      1
-29.0      1
-28.0      3
          ..
 989.0     1
 1046.0    1
 1088.0    1
 1165.0    1
 1202.0    1
Length: 473, dtype: int64

In [0]:
df = df.loc[~df.CANCELLED.isin([1])]

In [0]:
df['OUTPUT_LABEL'] = (df.DEPARTURE_DELAY >= 0).astype('int')

In [19]:
df_airports = pd.read_csv('airports.csv')
df_airports.head()

,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
0,ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.65236,-75.44040
1,ABI,Abilene Regional Airport,Abilene,TX,USA,32.41132,-99.68190
2,ABQ,Albuquerque International Sunport,Albuquerque,NM,USA,35.04022,-106.60919
3,ABR,Aberdeen Regional Airport,Aberdeen,SD,USA,45.44906,-98.42183
4,ABY,Southwest Georgia Regional Airport,Albany,GA,USA,31.53552,-84.19447


In [0]:
df.loc[~df.ORIGIN_AIRPORT.isin(df_airports.IATA_CODE.values), 'ORIGIN_AIRPORT'] = 'OTHER'
df.loc[~df.DESTINATION_AIRPORT.isin(df_airports.IATA_CODE.values), 'DESTINATION_AIRPORT'] = 'OTHER'

In [21]:
cols_num = ['SCHEDULED_DEPARTURE', 'SCHEDULED_TIME', 'DISTANCE', 'SCHEDULED_ARRIVAL', 'DAY_OF_WEEK']
cols_num
df[cols_num].isnull().sum()

SCHEDULED_DEPARTURE    0
SCHEDULED_TIME         0
DISTANCE               0
SCHEDULED_ARRIVAL      0
DAY_OF_WEEK            0
dtype: int64

In [22]:
cols_cat = ['AIRLINE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT']
cols_cat
df[cols_cat].isnull().sum()

AIRLINE                0
ORIGIN_AIRPORT         0
DESTINATION_AIRPORT    0
dtype: int64

In [23]:
df[cols_cat].nunique()

AIRLINE                 14
ORIGIN_AIRPORT         321
DESTINATION_AIRPORT    319
dtype: int64

In [0]:
df_cat = pd.get_dummies(df[cols_cat], drop_first=True)

In [25]:
df_cat.head()

,AIRLINE_AS,AIRLINE_B6,AIRLINE_DL,AIRLINE_EV,AIRLINE_F9,AIRLINE_HA,AIRLINE_MQ,AIRLINE_NK,AIRLINE_OO,AIRLINE_UA,AIRLINE_US,AIRLINE_VX,AIRLINE_WN,ORIGIN_AIRPORT_ABI,ORIGIN_AIRPORT_ABQ,ORIGIN_AIRPORT_ABR,ORIGIN_AIRPORT_ABY,ORIGIN_AIRPORT_ACK,ORIGIN_AIRPORT_ACT,ORIGIN_AIRPORT_ACV,ORIGIN_AIRPORT_ACY,ORIGIN_AIRPORT_ADK,ORIGIN_AIRPORT_ADQ,ORIGIN_AIRPORT_AEX,ORIGIN_AIRPORT_AGS,ORIGIN_AIRPORT_AKN,ORIGIN_AIRPORT_ALB,ORIGIN_AIRPORT_ALO,ORIGIN_AIRPORT_AMA,ORIGIN_AIRPORT_ANC,ORIGIN_AIRPORT_APN,ORIGIN_AIRPORT_ASE,ORIGIN_AIRPORT_ATL,ORIGIN_AIRPORT_ATW,ORIGIN_AIRPORT_AUS,ORIGIN_AIRPORT_AVL,ORIGIN_AIRPORT_AVP,ORIGIN_AIRPORT_AZO,ORIGIN_AIRPORT_BDL,ORIGIN_AIRPORT_BET,...,DESTINATION_AIRPORT_SIT,DESTINATION_AIRPORT_SJC,DESTINATION_AIRPORT_SJT,DESTINATION_AIRPORT_SJU,DESTINATION_AIRPORT_SLC,DESTINATION_AIRPORT_SMF,DESTINATION_AIRPORT_SMX,DESTINATION_AIRPORT_SNA,DESTINATION_AIRPORT_SPI,DESTINATION_AIRPORT_SPS,DESTINATION_AIRPORT_SRQ,DESTINATION_AIRPORT_STC,DESTINATION_AIRPORT_STL,DESTINATION_AIRPORT_STT,DESTINATION_AIRPORT_STX,DESTINATION_AIRPORT_SUN,DESTINATION_AIRPORT_SUX,DESTINATION_AIRPORT_SWF,DESTINATION_AIRPORT_SYR,DESTINATION_AIRPORT_TLH,DESTINATION_AIRPORT_TOL,DESTINATION_AIRPORT_TPA,DESTINATION_AIRPORT_TRI,DESTINATION_AIRPORT_TTN,DESTINATION_AIRPORT_TUL,DESTINATION_AIRPORT_TUS,DESTINATION_AIRPORT_TVC,DESTINATION_AIRPORT_TWF,DESTINATION_AIRPORT_TXK,DESTINATION_AIRPORT_TYR,DESTINATION_AIRPORT_TYS,DESTINATION_AIRPORT_UST,DESTINATION_AIRPORT_VEL,DESTINATION_AIRPORT_VLD,DESTINATION_AIRPORT_VPS,DESTINATION_AIRPORT_WRG,DESTINATION_AIRPORT_WYS,DESTINATION_AIRPORT_XNA,DESTINATION_AIRPORT_YAK,DESTINATION_AIRPORT_YUM
1120964,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2536963,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4034065,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4632793,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4857038,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
df = pd.concat([df, df_cat], axis = 1)

In [27]:
cols_all_cat = list(df_cat.columns)
print('number of categorical: ', len(cols_all_cat))

number of categorical:  651


In [28]:
cols_input = cols_num + cols_all_cat
df_data = df[cols_input + ['OUTPUT_LABEL']]
df_data.head()

,SCHEDULED_DEPARTURE,SCHEDULED_TIME,DISTANCE,SCHEDULED_ARRIVAL,DAY_OF_WEEK,AIRLINE_AS,AIRLINE_B6,AIRLINE_DL,AIRLINE_EV,AIRLINE_F9,AIRLINE_HA,AIRLINE_MQ,AIRLINE_NK,AIRLINE_OO,AIRLINE_UA,AIRLINE_US,AIRLINE_VX,AIRLINE_WN,ORIGIN_AIRPORT_ABI,ORIGIN_AIRPORT_ABQ,ORIGIN_AIRPORT_ABR,ORIGIN_AIRPORT_ABY,ORIGIN_AIRPORT_ACK,ORIGIN_AIRPORT_ACT,ORIGIN_AIRPORT_ACV,ORIGIN_AIRPORT_ACY,ORIGIN_AIRPORT_ADK,ORIGIN_AIRPORT_ADQ,ORIGIN_AIRPORT_AEX,ORIGIN_AIRPORT_AGS,ORIGIN_AIRPORT_AKN,ORIGIN_AIRPORT_ALB,ORIGIN_AIRPORT_ALO,ORIGIN_AIRPORT_AMA,ORIGIN_AIRPORT_ANC,ORIGIN_AIRPORT_APN,ORIGIN_AIRPORT_ASE,ORIGIN_AIRPORT_ATL,ORIGIN_AIRPORT_ATW,ORIGIN_AIRPORT_AUS,...,DESTINATION_AIRPORT_SJC,DESTINATION_AIRPORT_SJT,DESTINATION_AIRPORT_SJU,DESTINATION_AIRPORT_SLC,DESTINATION_AIRPORT_SMF,DESTINATION_AIRPORT_SMX,DESTINATION_AIRPORT_SNA,DESTINATION_AIRPORT_SPI,DESTINATION_AIRPORT_SPS,DESTINATION_AIRPORT_SRQ,DESTINATION_AIRPORT_STC,DESTINATION_AIRPORT_STL,DESTINATION_AIRPORT_STT,DESTINATION_AIRPORT_STX,DESTINATION_AIRPORT_SUN,DESTINATION_AIRPORT_SUX,DESTINATION_AIRPORT_SWF,DESTINATION_AIRPORT_SYR,DESTINATION_AIRPORT_TLH,DESTINATION_AIRPORT_TOL,DESTINATION_AIRPORT_TPA,DESTINATION_AIRPORT_TRI,DESTINATION_AIRPORT_TTN,DESTINATION_AIRPORT_TUL,DESTINATION_AIRPORT_TUS,DESTINATION_AIRPORT_TVC,DESTINATION_AIRPORT_TWF,DESTINATION_AIRPORT_TXK,DESTINATION_AIRPORT_TYR,DESTINATION_AIRPORT_TYS,DESTINATION_AIRPORT_UST,DESTINATION_AIRPORT_VEL,DESTINATION_AIRPORT_VLD,DESTINATION_AIRPORT_VPS,DESTINATION_AIRPORT_WRG,DESTINATION_AIRPORT_WYS,DESTINATION_AIRPORT_XNA,DESTINATION_AIRPORT_YAK,DESTINATION_AIRPORT_YUM,OUTPUT_LABEL
1120964,1642,58.0,179,1840,6,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2536963,610,185.0,1236,1015,3,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4034065,1155,100.0,544,1335,2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4632793,1345,155.0,930,1720,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4857038,1819,179.0,1182,2218,5,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [0]:
df_data = df_data.sample(n = len(df_data))
df_data = df_data.reset_index(drop = True)

In [30]:
df_valid_test = df_data.sample(frac=0.30)
print('Split size: %.3f' %(len(df_valid_test)/len(df_data)))

Split size: 0.300


In [0]:
df_test = df_valid_test.sample(frac = 0.5)
df_valid = df_valid_test.drop(df_test.index)

In [0]:
df_train_all = df_data.drop(df_valid_test.index)

In [33]:
print('Test n = %d' %(len(df_test)))
print('Valid n = %d' %(len(df_valid)))
print('Train n = %d' %(len(df_train_all)))
df_test

Test n = 14776
Valid n = 14775
Train n = 68951


,SCHEDULED_DEPARTURE,SCHEDULED_TIME,DISTANCE,SCHEDULED_ARRIVAL,DAY_OF_WEEK,AIRLINE_AS,AIRLINE_B6,AIRLINE_DL,AIRLINE_EV,AIRLINE_F9,AIRLINE_HA,AIRLINE_MQ,AIRLINE_NK,AIRLINE_OO,AIRLINE_UA,AIRLINE_US,AIRLINE_VX,AIRLINE_WN,ORIGIN_AIRPORT_ABI,ORIGIN_AIRPORT_ABQ,ORIGIN_AIRPORT_ABR,ORIGIN_AIRPORT_ABY,ORIGIN_AIRPORT_ACK,ORIGIN_AIRPORT_ACT,ORIGIN_AIRPORT_ACV,ORIGIN_AIRPORT_ACY,ORIGIN_AIRPORT_ADK,ORIGIN_AIRPORT_ADQ,ORIGIN_AIRPORT_AEX,ORIGIN_AIRPORT_AGS,ORIGIN_AIRPORT_AKN,ORIGIN_AIRPORT_ALB,ORIGIN_AIRPORT_ALO,ORIGIN_AIRPORT_AMA,ORIGIN_AIRPORT_ANC,ORIGIN_AIRPORT_APN,ORIGIN_AIRPORT_ASE,ORIGIN_AIRPORT_ATL,ORIGIN_AIRPORT_ATW,ORIGIN_AIRPORT_AUS,...,DESTINATION_AIRPORT_SJC,DESTINATION_AIRPORT_SJT,DESTINATION_AIRPORT_SJU,DESTINATION_AIRPORT_SLC,DESTINATION_AIRPORT_SMF,DESTINATION_AIRPORT_SMX,DESTINATION_AIRPORT_SNA,DESTINATION_AIRPORT_SPI,DESTINATION_AIRPORT_SPS,DESTINATION_AIRPORT_SRQ,DESTINATION_AIRPORT_STC,DESTINATION_AIRPORT_STL,DESTINATION_AIRPORT_STT,DESTINATION_AIRPORT_STX,DESTINATION_AIRPORT_SUN,DESTINATION_AIRPORT_SUX,DESTINATION_AIRPORT_SWF,DESTINATION_AIRPORT_SYR,DESTINATION_AIRPORT_TLH,DESTINATION_AIRPORT_TOL,DESTINATION_AIRPORT_TPA,DESTINATION_AIRPORT_TRI,DESTINATION_AIRPORT_TTN,DESTINATION_AIRPORT_TUL,DESTINATION_AIRPORT_TUS,DESTINATION_AIRPORT_TVC,DESTINATION_AIRPORT_TWF,DESTINATION_AIRPORT_TXK,DESTINATION_AIRPORT_TYR,DESTINATION_AIRPORT_TYS,DESTINATION_AIRPORT_UST,DESTINATION_AIRPORT_VEL,DESTINATION_AIRPORT_VLD,DESTINATION_AIRPORT_VPS,DESTINATION_AIRPORT_WRG,DESTINATION_AIRPORT_WYS,DESTINATION_AIRPORT_XNA,DESTINATION_AIRPORT_YAK,DESTINATION_AIRPORT_YUM,OUTPUT_LABEL
71889,1440,85.0,407,1605,2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
53227,615,256.0,1562,931,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
144,850,126.0,641,956,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
53009,840,90.0,391,1010,5,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
84028,1004,111.0,552,1155,2,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1293,825,75.0,255,840,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
87103,1000,115.0,581,1155,4,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
65104,1045,243.0,1616,1248,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
72444,2225,80.0,361,2345,5,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [34]:
rows_pos = df_train_all.OUTPUT_LABEL == 1
df_train_pos = df_train_all.loc[rows_pos]
df_train_neg = df_train_all.loc[~rows_pos]

df_train = pd.concat([df_train_pos, df_train_neg.sample(n=len(df_train_pos))], axis=0)

df_train = df_train.sample(n=len(df_train)).reset_index(drop=True)

print('Train balanced prevalence(n= %d)'%(len(df_train)))

Train balanced prevalence(n= 59024)


In [35]:
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import accuracy_score
import lightgbm as lgb
y = df_data.OUTPUT_LABEL
X = df_data.drop(['OUTPUT_LABEL'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

dtrain=xgb.DMatrix(X_train,label=y_train)
dtest=xgb.DMatrix(X_test)
parameters={'max_depth':7, 'silent':1,'objective':'binary:logistic','eval_metric':'auc','learning_rate':.05}

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


In [0]:
num_round=50
from datetime import datetime 
start = datetime.now() 
xg=xgb.train(parameters,dtrain,num_round) 
stop = datetime.now()

In [37]:
execution_time_xgb = stop-start 
execution_time_xgb

datetime.timedelta(0, 81, 464530)

In [0]:
ypred1=xg.predict(dtest)

In [0]:
for i in range(0,19685): 
    if ypred1[i]>=.5:       # setting threshold to .5 
       ypred1[i]=1 
    else: 
       ypred1[i]=0
ypred2=np.int64(ypred1)
ypred = pd.Series(ypred2)

In [42]:
#calculating accuracy of our model 
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix 

accuracy_xgb = accuracy_score(y_test,ypred)
results1 = confusion_matrix(y_test, ypred)  
print(accuracy_xgb)
print(results1)

0.6335211410588295
[[9272 2090]
 [5130 3209]]


In [0]:
import lightgbm as lgbm
train_data=lgbm.Dataset(X_train,label=y_train)

In [0]:
#setting parameters for lightgbm
param = {'num_leaves':150, 'objective':'binary','max_depth':7,'learning_rate':.05,'max_bin':200}
param['metric'] = ['auc', 'binary_logloss']

In [0]:
#training our model using light gbm
num_round=50
start=datetime.now()
lgbm=lgb.train(param,train_data,num_round)
stop=datetime.now()

In [46]:
execution_time_lgbm = stop-start
execution_time_lgbm

datetime.timedelta(0, 1, 695878)

In [0]:
ypred2=lgbm.predict(X_test)

In [0]:
for i in range(0,9769):
    if ypred2[i]>=.5:       # setting threshold to .5
       ypred2[i]=1
    else:  
       ypred2[i]=0

ypred22=np.int64(ypred2)
ypred23 = pd.Series(ypred22)

In [52]:
accuracy_lgbm = accuracy_score(y_test, ypred23)
results2 = confusion_matrix(y_test, ypred23)  
print(results2)
print(accuracy_lgbm)

[[10346  1016]
 [ 6853  1486]]
0.6005786508299071
